In [1]:
!pip install pyserial


In [2]:
import serial

print("Pyserial successfully imported!")


Pyserial successfully imported!


In [3]:
import random
import time

class SimulatedSensor:
    def __init__(self, initial_value=50, min_value=0, max_value=100):
        self.value = initial_value  # Başlangıç değeri
        self.min_value = min_value  # Minimum sınır
        self.max_value = max_value  # Maksimum sınır
        self.trend = 0  # Sensör değişim yönü (sabitlik veya dalgalanma için)

    def read(self):
        """Sensör verisini oku ve rastgele bir değişim uygula."""
        # Rastgele bir "gürültü" ekle
        noise = random.uniform(-5, 5)
        
        # Trend ile birleştir
        self.trend += random.uniform(-1, 1)  # Trend değişimi
        self.trend = max(-2, min(2, self.trend))  # Trend sınırlandırması
        
        # Yeni değer hesapla
        self.value += self.trend + noise
        
        # Değer sınırlandırma
        self.value = max(self.min_value, min(self.max_value, self.value))
        
        return self.value

# Sensör simülasyonu
sensor = SimulatedSensor()

# Sürekli veri üretme
try:
    while True:
        measured_value = sensor.read()  # Sensör verisi al
        print(f"Measured Distance: {measured_value:.2f}")
        time.sleep(0.1)  # 100 ms'lik bir okuma süresi (tipik bir sensör için)
except KeyboardInterrupt:
    print("Simulation stopped.")


Measured Distance: 47.72
Measured Distance: 46.60
Measured Distance: 48.15
Measured Distance: 52.20
Measured Distance: 55.35
Measured Distance: 60.49
Simulation stopped.


In [4]:
import random
import time

class SimulatedSensor:
    def __init__(self, initial_value=50, min_value=0, max_value=100):
        self.value = initial_value  # Başlangıç değeri
        self.min_value = min_value  # Minimum sınır
        self.max_value = max_value  # Maksimum sınır
        self.prev_value = initial_value  # Önceki mesafe
        self.prev_time = time.time()  # Başlangıç zamanı

    def read(self):
        """Sensör verisini oku ve rastgele bir değişim uygula."""
        # Rastgele bir "gürültü" ekle
        noise = random.uniform(-5, 5)
        
        # Mesafe değerini güncelle
        self.value += random.uniform(-1, 1) + noise
        self.value = max(self.min_value, min(self.max_value, self.value))  # Sınırlandır
        
        return self.value

    def get_speed(self):
        """Hızı mesafe farkı ve zaman farkı ile hesapla."""
        current_time = time.time()
        delta_time = current_time - self.prev_time  # Zaman farkı
        delta_distance = self.value - self.prev_value  # Mesafe farkı
        
        if delta_time > 0:  # Zaman farkı sıfır değilse
            speed = delta_distance / delta_time  # Hızı hesapla
        else:
            speed = 0  # Zaman farkı sıfırsa hız sıfır kabul edilir

        self.prev_value = self.value  # Önceki mesafeyi güncelle
        self.prev_time = current_time  # Önceki zamanı güncelle

        return speed

# PI Kontrolcüsü
class PIController:
    def __init__(self, target_speed, Kp, Ki):
        self.target_speed = target_speed  # Hedef hız (50 km/h)
        self.Kp = Kp  # Proportional (P) kazancı
        self.Ki = Ki  # Integral (I) kazancı
        self.prev_error = 0  # Önceki hata
        self.integral = 0  # Hata birikimi (integral)

    def control(self, current_speed):
        """PI kontrolünü uygula."""
        error = self.target_speed - current_speed  # Hata hesaplama
        self.integral += error  # Hata birikimi
        # Proportional ve Integral terimlerinin toplamı
        control_signal = self.Kp * error + self.Ki * self.integral
        return control_signal


# Sensör ve PI Kontrolcüsü nesneleri oluşturuluyor
sensor = SimulatedSensor(initial_value=50)  # Başlangıç değeri 50 cm
pi_controller = PIController(target_speed=50, Kp=0.1, Ki=0.05)  # Hedef hız 50 km/h

# Sürekli veri üretme ve PI kontrolü uygulama
try:
    while True:
        measured_distance = sensor.read()  # Sensör verisi al
        current_speed = sensor.get_speed()  # Hızı hesapla
        
        # Kontrolcü tarafından hız ayarı yapılır
        control_signal = pi_controller.control(current_speed)
        
        # Kontrol sinyali ile mesafeyi düzelt
        corrected_speed = current_speed + control_signal
        
        # Hedef hıza ulaşmak için düzeltmeler yapılır
        print(f"Measured Distance: {measured_distance:.2f} cm | Current Speed: {current_speed:.2f} cm/s | Corrected Speed: {corrected_speed:.2f} cm/s")
        
        time.sleep(0.1)  # 100 ms bekleme

except KeyboardInterrupt:
    print("Simulation stopped.")


Measured Distance: 48.62 cm | Current Speed: 0.00 cm/s | Corrected Speed: 7.50 cm/s
Measured Distance: 46.36 cm | Current Speed: -22.44 cm/s | Corrected Speed: -9.07 cm/s
Measured Distance: 47.32 cm | Current Speed: 9.48 cm/s | Corrected Speed: 21.68 cm/s
Measured Distance: 44.20 cm | Current Speed: -31.02 cm/s | Corrected Speed: -10.72 cm/s
Measured Distance: 41.88 cm | Current Speed: -23.06 cm/s | Corrected Speed: 0.10 cm/s
Measured Distance: 44.34 cm | Current Speed: 24.44 cm/s | Corrected Speed: 44.12 cm/s
Measured Distance: 40.40 cm | Current Speed: -39.10 cm/s | Corrected Speed: -8.60 cm/s
Measured Distance: 38.36 cm | Current Speed: -20.39 cm/s | Corrected Speed: 11.75 cm/s
Measured Distance: 38.43 cm | Current Speed: 0.69 cm/s | Corrected Speed: 33.19 cm/s
Measured Distance: 37.29 cm | Current Speed: -11.34 cm/s | Corrected Speed: 25.44 cm/s
Measured Distance: 39.95 cm | Current Speed: 26.47 cm/s | Corrected Speed: 60.64 cm/s
Measured Distance: 38.54 cm | Current Speed: -14.02 

In [5]:
import random
import time

# Simüle edilmiş sensör
class SimulatedSensor:
    def __init__(self, initial_value=50, min_value=0, max_value=100):
        self.value = initial_value  # Başlangıç mesafesi
        self.min_value = min_value  # Minimum mesafe
        self.max_value = max_value  # Maksimum mesafe
        self.prev_value = initial_value  # Önceki mesafe
        self.prev_time = time.time()  # Başlangıç zamanı

    def read(self):
        """Sensör verisini oku ve rastgele bir değişim uygula."""
        noise = random.uniform(-5, 5)  # Gürültü
        self.value += random.uniform(-1, 1) + noise
        self.value = max(self.min_value, min(self.max_value, self.value))  # Sınırlandırma
        return self.value

    def get_speed(self):
        """Hızı, mesafe ve zaman farkına göre hesapla."""
        current_time = time.time()
        delta_time = current_time - self.prev_time  # Zaman farkı
        delta_distance = self.value - self.prev_value  # Mesafe farkı

        if delta_time > 0:
            speed = delta_distance / delta_time
        else:
            speed = 0

        self.prev_value = self.value
        self.prev_time = current_time
        return speed


# PI Kontrolcüsü
class PIController:
    def __init__(self, target_speed, Kp, Ki, Kd=0.05):
        self.target_speed = target_speed  # Hedef hız
        self.Kp = Kp  # Proportional kazancı
        self.Ki = Ki  # Integral kazancı
        self.Kd = Kd  # Derinlik kontrolü için mesafe kazancı
        self.integral = 0  # Hata birikimi

    def control(self, current_speed, measured_distance, target_distance=50):
        """Hem hız hem mesafeye dayalı PI kontrol."""
        # Hız hatası
        speed_error = self.target_speed - current_speed
        # Mesafe hatası
        distance_error = target_distance - measured_distance

        # Hata birikimi (integral terimi)
        self.integral += speed_error

        # Kontrol sinyali
        control_signal = (
            self.Kp * speed_error +
            self.Ki * self.integral +
            self.Kd * distance_error
        )
        return control_signal


# Simülasyon
sensor = SimulatedSensor(initial_value=50)  # Başlangıç mesafesi 50 cm
pi_controller = PIController(target_speed=0, Kp=0.1, Ki=0.05, Kd=0.02)  # Hedef hız 0 cm/s (durgunluk)

try:
    while True:
        # Sensör verisi al
        measured_distance = sensor.read()
        # Hızı hesapla
        current_speed = sensor.get_speed()

        # PI kontrolünü uygula
        control_signal = pi_controller.control(
            current_speed=current_speed,
            measured_distance=measured_distance,
            target_distance=50  # Hedef mesafe
        )

        # Kontrol sinyali ile hız düzeltilir
        corrected_speed = current_speed + control_signal

        # Durum bilgisi yazdır
        print(
            f"Measured Distance: {measured_distance:.2f} cm | "
            f"Current Speed: {current_speed:.2f} cm/s | "
            f"Corrected Speed: {corrected_speed:.2f} cm/s"
        )

        # Bekleme (100 ms)
        time.sleep(0.1)

except KeyboardInterrupt:
    print("Simulation stopped.")


Measured Distance: 49.42 cm | Current Speed: 0.00 cm/s | Corrected Speed: 0.01 cm/s
Measured Distance: 47.35 cm | Current Speed: -20.50 cm/s | Corrected Speed: -17.37 cm/s
Measured Distance: 45.94 cm | Current Speed: -14.03 cm/s | Corrected Speed: -10.82 cm/s
Measured Distance: 48.04 cm | Current Speed: 20.91 cm/s | Corrected Speed: 19.54 cm/s
Measured Distance: 45.46 cm | Current Speed: -25.65 cm/s | Corrected Speed: -21.03 cm/s
Measured Distance: 47.85 cm | Current Speed: 23.87 cm/s | Corrected Speed: 22.29 cm/s
Measured Distance: 51.72 cm | Current Speed: 38.47 cm/s | Corrected Speed: 33.43 cm/s
Measured Distance: 53.28 cm | Current Speed: 15.48 cm/s | Corrected Speed: 11.94 cm/s
Measured Distance: 50.20 cm | Current Speed: -30.70 cm/s | Corrected Speed: -28.03 cm/s
Measured Distance: 50.89 cm | Current Speed: 6.86 cm/s | Corrected Speed: 5.42 cm/s
Measured Distance: 53.08 cm | Current Speed: 21.89 cm/s | Corrected Speed: 17.81 cm/s
Measured Distance: 49.04 cm | Current Speed: -40.3

In [6]:
import random
import time

# Simüle edilmiş sensör
class SimulatedSensor:
    def __init__(self, initial_value=50, min_value=0, max_value=100):
        self.value = initial_value  # Başlangıç mesafesi
        self.min_value = min_value  # Minimum mesafe
        self.max_value = max_value  # Maksimum mesafe
        self.prev_value = initial_value  # Önceki mesafe
        self.prev_time = time.time()  # Başlangıç zamanı

    def read(self):
        """Sensör verisini oku ve rastgele bir değişim uygula."""
        noise = random.uniform(-5, 5)  # Gürültü
        self.value += random.uniform(-1, 1) + noise
        self.value = max(self.min_value, min(self.max_value, self.value))  # Sınırlandırma
        return self.value

    def get_speed(self):
        """Hızı, mesafe ve zaman farkına göre hesapla."""
        current_time = time.time()
        delta_time = current_time - self.prev_time  # Zaman farkı
        delta_distance = self.value - self.prev_value  # Mesafe farkı

        if delta_time > 0:
            speed = delta_distance / delta_time
        else:
            speed = 0

        self.prev_value = self.value
        self.prev_time = current_time
        return speed


# PI Kontrolcüsü
class PIController:
    def __init__(self, target_speed, Kp, Ki, Kd=0.05):
        self.target_speed = target_speed  # Hedef hız
        self.Kp = Kp  # Proportional kazancı
        self.Ki = Ki  # Integral kazancı
        self.Kd = Kd  # Mesafe kazancı
        self.integral = 0  # Hata birikimi

    def control(self, current_speed, measured_distance, target_distance=50):
        """Hem hız hem mesafeye dayalı PI kontrol."""
        # Hız hatası
        speed_error = self.target_speed - current_speed
        # Mesafe hatası
        distance_error = target_distance - measured_distance

        # Mantıksal kontrol: Mesafe hatasına göre hız yönünü düzelt
        if measured_distance > target_distance and current_speed > 0:
            # Mesafe fazla ve hız pozitifse, düzeltici hız negatif olmalı
            speed_error = -abs(speed_error)
        elif measured_distance < target_distance and current_speed < 0:
            # Mesafe az ve hız negatifse, düzeltici hız pozitif olmalı
            speed_error = abs(speed_error)

        # Hata birikimi (integral terimi)
        self.integral += speed_error

        # Kontrol sinyali
        control_signal = (
            self.Kp * speed_error +
            self.Ki * self.integral +
            self.Kd * distance_error
        )
        return control_signal


# Simülasyon
sensor = SimulatedSensor(initial_value=50)  # Başlangıç mesafesi 50 cm
pi_controller = PIController(target_speed=0, Kp=0.1, Ki=0.05, Kd=0.02)  # Hedef hız 0 cm/s (durgunluk)

try:
    while True:
        # Sensör verisi al
        measured_distance = sensor.read()
        # Hızı hesapla
        current_speed = sensor.get_speed()

        # PI kontrolünü uygula
        control_signal = pi_controller.control(
            current_speed=current_speed,
            measured_distance=measured_distance,
            target_distance=50  # Hedef mesafe
        )

        # Kontrol sinyali ile hız düzeltilir
        corrected_speed = current_speed + control_signal

        # Durum bilgisi yazdır
        print(
            f"Measured Distance: {measured_distance:.2f} cm | "
            f"Current Speed: {current_speed:.2f} cm/s | "
            f"Corrected Speed: {corrected_speed:.2f} cm/s"
        )

        # Bekleme (100 ms)
        time.sleep(0.1)

except KeyboardInterrupt:
    print("Simulation stopped.")


Measured Distance: 46.32 cm | Current Speed: 0.00 cm/s | Corrected Speed: 0.07 cm/s
Measured Distance: 45.22 cm | Current Speed: -10.88 cm/s | Corrected Speed: -9.15 cm/s
Measured Distance: 43.83 cm | Current Speed: -13.85 cm/s | Corrected Speed: -11.10 cm/s
Measured Distance: 40.77 cm | Current Speed: -30.43 cm/s | Corrected Speed: -24.44 cm/s
Measured Distance: 39.75 cm | Current Speed: -10.17 cm/s | Corrected Speed: -5.68 cm/s
Measured Distance: 41.75 cm | Current Speed: 19.90 cm/s | Corrected Speed: 20.35 cm/s
Measured Distance: 40.25 cm | Current Speed: -14.94 cm/s | Corrected Speed: -10.23 cm/s
Measured Distance: 45.67 cm | Current Speed: 53.78 cm/s | Corrected Speed: 48.82 cm/s
Measured Distance: 44.35 cm | Current Speed: -13.16 cm/s | Corrected Speed: -10.75 cm/s
Measured Distance: 45.46 cm | Current Speed: 11.01 cm/s | Corrected Speed: 10.43 cm/s
Measured Distance: 49.43 cm | Current Speed: 39.55 cm/s | Corrected Speed: 34.07 cm/s
Measured Distance: 45.67 cm | Current Speed: -

In [16]:
import random
import time

# Simüle edilmiş sensör
class SimulatedSensor:
    def __init__(self, initial_value=50, min_value=0, max_value=100):
        self.value = initial_value  # Başlangıç mesafesi
        self.min_value = min_value  # Minimum mesafe
        self.max_value = max_value  # Maksimum mesafe
        self.prev_value = initial_value  # Önceki mesafe
        self.prev_time = time.time()  # Başlangıç zamanı

    def read(self):
        """Sensör verisini oku ve rastgele bir değişim uygula."""
        noise = random.uniform(-5, 5)  # Gürültü
        self.value += random.uniform(-1, 1) + noise
        self.value = max(self.min_value, min(self.max_value, self.value))  # Sınırlandırma
        return self.value

    def get_speed(self):
        """Hızı, mesafe ve zaman farkına göre hesapla."""
        current_time = time.time()
        delta_time = current_time - self.prev_time  # Zaman farkı
        delta_distance = self.value - self.prev_value  # Mesafe farkı

        if delta_time > 0:
            speed = delta_distance / delta_time
        else:
            speed = 0

        self.prev_value = self.value
        self.prev_time = current_time
        return speed


# PI Kontrolcüsü
class PIController:
    def __init__(self, target_speed, Kp, Ki, Kd=0.05):
        self.target_speed = target_speed  # Hedef hız
        self.Kp = Kp  # Proportional kazancı
        self.Ki = Ki  # Integral kazancı
        self.Kd = Kd  # Mesafe kazancı
        self.integral = 0  # Hata birikimi

    def control(self, current_speed, measured_distance, target_distance=50):
        """Hedef mesafeye ulaşmak için PI kontrol sinyali üret."""
        # Mesafe hatası
        distance_error = target_distance - measured_distance
        
        # Mesafeye göre hız yönlendirmesi
        if distance_error > 0:
            # Hedefe yaklaşmak için hız pozitif olmalı
            speed_error = abs(self.target_speed - current_speed)
        else:
            # Hedefe yaklaşmak için hız negatif olmalı
            speed_error = -abs(self.target_speed - current_speed)

        # Hata birikimi (integral terimi)
        self.integral += speed_error

        # Kontrol sinyali
        control_signal = (
            self.Kp * speed_error +  # Hız hatası katkısı
            self.Ki * self.integral +  # Hata birikimi katkısı
            self.Kd * distance_error   # Mesafe hatası katkısı
        )

        return control_signal


# Simülasyon
sensor = SimulatedSensor(initial_value=70)  # Başlangıç mesafesi 70 cm
pi_controller = PIController(target_speed=0, Kp=0.1, Ki=0.05, Kd=0.02)  # Hedef hız 0 cm/s (durgunluk)

try:
    while True:
        # Sensör verisi al
        measured_distance = sensor.read()
        # Hızı hesapla
        current_speed = sensor.get_speed()

        # PI kontrolünü uygula
        control_signal = pi_controller.control(
            current_speed=current_speed,
            measured_distance=measured_distance,
            target_distance=50  # Hedef mesafe
        )

        # Kontrol sinyali ile hız düzeltilir
        corrected_speed = current_speed + control_signal

        # Durum bilgisi yazdır
        print(
            f"Measured Distance: {measured_distance:.2f} cm | "
            f"Current Speed: {current_speed:.2f} cm/s | "
            f"Corrected Speed: {corrected_speed:.2f} cm/s",
            flush=True
        )

        # Bekleme (100 ms)
        time.sleep(0.1)

except KeyboardInterrupt:
    print("Simulation stopped.")


Measured Distance: 73.07 cm | Current Speed: 0.00 cm/s | Corrected Speed: -0.46 cm/s
Measured Distance: 70.85 cm | Current Speed: -21.94 cm/s | Corrected Speed: -25.64 cm/s
Measured Distance: 73.52 cm | Current Speed: 26.41 cm/s | Corrected Speed: 20.88 cm/s
Measured Distance: 74.09 cm | Current Speed: 5.68 cm/s | Corrected Speed: 1.93 cm/s
Measured Distance: 71.64 cm | Current Speed: -23.93 cm/s | Corrected Speed: -30.65 cm/s
Measured Distance: 72.78 cm | Current Speed: 11.19 cm/s | Corrected Speed: 5.16 cm/s
Measured Distance: 69.86 cm | Current Speed: -28.98 cm/s | Corrected Speed: -38.18 cm/s
Measured Distance: 68.66 cm | Current Speed: -11.83 cm/s | Corrected Speed: -19.89 cm/s
Measured Distance: 63.74 cm | Current Speed: -48.42 cm/s | Corrected Speed: -62.45 cm/s
Measured Distance: 62.48 cm | Current Speed: -12.40 cm/s | Corrected Speed: -23.42 cm/s
Measured Distance: 59.19 cm | Current Speed: -32.30 cm/s | Corrected Speed: -46.87 cm/s
Measured Distance: 56.76 cm | Current Speed:

In [21]:
import random
import time

# Simüle edilmiş sensör
class SimulatedSensor:
    def __init__(self, initial_value=50, min_value=0, max_value=100):
        self.value = initial_value  # Başlangıç mesafesi
        self.min_value = min_value  # Minimum mesafe
        self.max_value = max_value  # Maksimum mesafe
        self.prev_value = initial_value  # Önceki mesafe
        self.prev_time = time.time()  # Başlangıç zamanı

    def read(self):
        """Sensör verisini oku ve rastgele bir değişim uygula."""
        noise = random.uniform(-5, 5)  # Gürültü
        self.value += random.uniform(-1, 1) + noise
        self.value = max(self.min_value, min(self.max_value, self.value))  # Sınırlandırma
        return self.value

    def get_speed(self):
        """Hızı, mesafe ve zaman farkına göre hesapla."""
        current_time = time.time()
        delta_time = current_time - self.prev_time  # Zaman farkı
        delta_distance = self.value - self.prev_value  # Mesafe farkı

        if delta_time > 0:
            speed = delta_distance / delta_time
        else:
            speed = 0

        self.prev_value = self.value
        self.prev_time = current_time
        return speed


# PI Kontrolcüsü
class PIController:
    def __init__(self, target_speed, Kp, Ki, Kd=0.05):
        self.target_speed = target_speed  # Hedef hız
        self.Kp = Kp  # Proportional kazancı
        self.Ki = Ki  # Integral kazancı
        self.Kd = Kd  # Mesafe kazancı
        self.integral = 0  # Hata birikimi

    def control(self, current_speed, measured_distance, target_distance=50):
        """Hedef mesafeye ulaşmak için PI kontrol sinyali üret."""
        # Mesafe hatası
        distance_error = target_distance - measured_distance
        
        # Mesafeye göre hız yönlendirmesi
        if distance_error > 0:
            # Hedefe yaklaşmak için hız pozitif olmalı
            speed_error = abs(self.target_speed - current_speed)
        else:
            # Hedefe yaklaşmak için hız negatif olmalı
            speed_error = -abs(self.target_speed - current_speed)

        # Hata birikimi (integral terimi)
        self.integral += speed_error

        # Kontrol sinyali
        control_signal = (
            self.Kp * speed_error +  # Hız hatası katkısı
            self.Ki * self.integral +  # Hata birikimi katkısı
            self.Kd * distance_error   # Mesafe hatası katkısı
        )

        return control_signal


# Simülasyon
sensor = SimulatedSensor(initial_value=70)  # Başlangıç mesafesi 70 cm
pi_controller = PIController(target_speed=0, Kp=0.3, Ki=0.25, Kd=0.08)  # Hedef hız 0 cm/s (durgunluk)

try:
    while True:
        # Sensör verisi al
        measured_distance = sensor.read()
        # Hızı hesapla
        current_speed = sensor.get_speed()

        # PI kontrolünü uygula
        control_signal = pi_controller.control(
            current_speed=current_speed,
            measured_distance=measured_distance,
            target_distance=50  # Hedef mesafe
        )

        # Kontrol sinyali ile hız düzeltilir
        corrected_speed = current_speed + control_signal

        # Durum bilgisi yazdır
        print(
            f"Measured Distance: {measured_distance:.2f} cm | "
            f"Current Speed: {current_speed:.2f} cm/s | "
            f"Corrected Speed: {corrected_speed:.2f} cm/s",
            flush=True
        )

        # Bekleme (100 ms)
        time.sleep(0.1)

except KeyboardInterrupt:
    print("Simulation stopped.")


Measured Distance: 72.32 cm | Current Speed: 0.00 cm/s | Corrected Speed: -1.79 cm/s
Measured Distance: 68.94 cm | Current Speed: -33.00 cm/s | Corrected Speed: -52.66 cm/s
Measured Distance: 72.21 cm | Current Speed: 32.16 cm/s | Corrected Speed: 4.44 cm/s
Measured Distance: 68.02 cm | Current Speed: -40.86 cm/s | Corrected Speed: -81.07 cm/s
Measured Distance: 66.64 cm | Current Speed: -13.63 cm/s | Corrected Speed: -48.96 cm/s
Measured Distance: 67.67 cm | Current Speed: 10.10 cm/s | Corrected Speed: -26.78 cm/s
Measured Distance: 63.53 cm | Current Speed: -40.67 cm/s | Corrected Speed: -96.56 cm/s
Measured Distance: 58.68 cm | Current Speed: -47.82 cm/s | Corrected Speed: -117.42 cm/s
Measured Distance: 55.22 cm | Current Speed: -34.19 cm/s | Corrected Speed: -107.96 cm/s
Measured Distance: 58.71 cm | Current Speed: 34.68 cm/s | Corrected Speed: -48.20 cm/s
Measured Distance: 59.24 cm | Current Speed: 5.19 cm/s | Corrected Speed: -70.18 cm/s
Measured Distance: 56.22 cm | Current Sp